In [20]:
import os
import shutil

# Ruta donde se encuentran todos los archivos
source_dir = './Dataset_500_EMPRESAS_v2'  # Cambia esto si es necesario

# Obtener todos los archivos en el directorio
files = os.listdir(source_dir)

# Crear un conjunto de todos los tickers únicos
tickers = set()
for file in files:
    if file.endswith('.csv') or file.endswith('.json'):
        parts = file.split('_')
        if len(parts) >= 2:
            ticker = parts[0]
            tickers.add(ticker)

# Crear carpetas y mover archivos
for ticker in tickers:
    folder_path = os.path.join(source_dir, ticker)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    for file in files:
        if file.startswith(ticker + '_'):
            source_file = os.path.join(source_dir, file)
            dest_file = os.path.join(folder_path, file)
            shutil.move(source_file, dest_file)
            print(f"Moved {file} to {folder_path}")

print("Organización completada.")

Moved FICO_completo.csv to ./Dataset_500_EMPRESAS_v2/FICO
Moved IQV_completo.csv to ./Dataset_500_EMPRESAS_v2/IQV
Moved PWR_completo.csv to ./Dataset_500_EMPRESAS_v2/PWR
Moved NFLX_completo.csv to ./Dataset_500_EMPRESAS_v2/NFLX
Moved DVN_completo.csv to ./Dataset_500_EMPRESAS_v2/DVN
Moved SHW_completo.csv to ./Dataset_500_EMPRESAS_v2/SHW
Moved LOW_completo.csv to ./Dataset_500_EMPRESAS_v2/LOW
Moved ACGL_completo.csv to ./Dataset_500_EMPRESAS_v2/ACGL
Moved HWM_completo.csv to ./Dataset_500_EMPRESAS_v2/HWM
Moved PYPL_completo.csv to ./Dataset_500_EMPRESAS_v2/PYPL
Moved ITW_completo.csv to ./Dataset_500_EMPRESAS_v2/ITW
Moved LYB_completo.csv to ./Dataset_500_EMPRESAS_v2/LYB
Moved TKO_completo.csv to ./Dataset_500_EMPRESAS_v2/TKO
Moved AFL_completo.csv to ./Dataset_500_EMPRESAS_v2/AFL
Moved BMY_completo.csv to ./Dataset_500_EMPRESAS_v2/BMY
Moved EXPD_completo.csv to ./Dataset_500_EMPRESAS_v2/EXPD
Moved RMD_completo.csv to ./Dataset_500_EMPRESAS_v2/RMD
Moved HBAN_completo.csv to ./Dataset_5

In [21]:
import pandas as pd
import os
import json

def procesar_ticker(ticker, base_path):
    # Cargar datos históricos
    hist_path = os.path.join(base_path, ticker, f"{ticker}_historico.csv")
    df_hist = pd.read_csv(hist_path, parse_dates=['Date'], index_col='Date')
    
    # Cargar datos financieros
    financial_dfs = []
    for tipo in ['cash_flow', 'balance_sheet', 'income_statement']:
        path = os.path.join(base_path, ticker, f"{ticker}_{tipo}.csv")
        if not os.path.exists(path):
            print(f"Advertencia: No existe {path}")
            continue
        df = pd.read_csv(path, index_col=0)
        df = df.T  # Transponer para que las fechas sean el índice
        df.index = pd.to_datetime(df.index)
        financial_dfs.append(df)
    
    if not financial_dfs:
        print(f"No hay datos financieros para {ticker}")
        return
    
    # Combinar datos financieros
    df_financial = pd.concat(financial_dfs, axis=1)
    
    # Crear índice diario covering el rango de fechas históricas
    start_date = df_hist.index.min()
    end_date = df_hist.index.max()
    idx_diario = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # Reindexar datos financieros al índice diario
    df_financial_diario = df_financial.reindex(idx_diario)
    
    # Forward fill para propagar valores
    df_financial_diario = df_financial_diario.ffill()
    
    # Rellenar con 0 las fechas anteriores al primer reporte financiero
    if not df_financial.empty:
        first_financial_date = df_financial.index.min()
        mask = idx_diario < first_financial_date
        df_financial_diario.loc[mask] = 0
    
    # Combinar con datos históricos
    df_combinado = df_hist.join(df_financial_diario, how='left')
    
    # Cargar sectorKey desde info.json
    info_path = os.path.join(base_path, ticker, f"{ticker}_info.json")
    with open(info_path, 'r') as f:
        info_data = json.load(f)
    sector_key = info_data.get('sectorKey', '')
    
    # Añadir sectorKey como columna
    df_combinado['sectorKey'] = sector_key
    
    # Guardar resultado
    output_path = os.path.join(base_path, f"{ticker}_completo.csv")
    df_combinado.to_csv(output_path, index_label='Date')
    print(f"Guardado: {output_path}")

# Procesar todos los tickers
base_path = './Dataset_500_EMPRESAS_v2'  # Cambia esto si es necesario
tickers = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]

for ticker in tickers:
    try:
        procesar_ticker(ticker, base_path)
    except Exception as e:
        print(f"Error procesando {ticker}: {str(e)}")

Guardado: ./Dataset_500_EMPRESAS_v2/CTAS_completo.csv
Guardado: ./Dataset_500_EMPRESAS_v2/WELL_completo.csv
Guardado: ./Dataset_500_EMPRESAS_v2/VZ_completo.csv
Guardado: ./Dataset_500_EMPRESAS_v2/AMZN_completo.csv
Guardado: ./Dataset_500_EMPRESAS_v2/CNP_completo.csv
Guardado: ./Dataset_500_EMPRESAS_v2/RCL_completo.csv
Guardado: ./Dataset_500_EMPRESAS_v2/CAT_completo.csv
Guardado: ./Dataset_500_EMPRESAS_v2/TFC_completo.csv
Guardado: ./Dataset_500_EMPRESAS_v2/AAPL_completo.csv
Guardado: ./Dataset_500_EMPRESAS_v2/BF-B_completo.csv
Guardado: ./Dataset_500_EMPRESAS_v2/PANW_completo.csv
Guardado: ./Dataset_500_EMPRESAS_v2/PM_completo.csv
Guardado: ./Dataset_500_EMPRESAS_v2/KHC_completo.csv
Guardado: ./Dataset_500_EMPRESAS_v2/TEL_completo.csv
Guardado: ./Dataset_500_EMPRESAS_v2/CMCSA_completo.csv
Guardado: ./Dataset_500_EMPRESAS_v2/GRMN_completo.csv
Guardado: ./Dataset_500_EMPRESAS_v2/ANET_completo.csv
Guardado: ./Dataset_500_EMPRESAS_v2/BRO_completo.csv
Guardado: ./Dataset_500_EMPRESAS_v2/CA

In [ ]:
import pandas as pd
import os
import numpy as np

def limpiar_datos(ticker, input_dir, output_dir):
    # Leer el archivo CSV
    input_path = os.path.join(input_dir, f"{ticker}_completo.csv")
    df = pd.read_csv(input_path)
    
    # Convertir la columna 'Free Cash Flow' a numérico, manejando errores
    df['Free Cash Flow'] = pd.to_numeric(df['Free Cash Flow'], errors='coerce')
    
    # Filtrar filas donde 'Free Cash Flow' no es NaN, vacío ni 0
    df_limpio = df[df['Free Cash Flow'].notna() & (df['Free Cash Flow'] != 0)]
    
    # Guardar el resultado
    output_path = os.path.join(output_dir, f"{ticker}_completo_arreglado.csv")
    df_limpio.to_csv(output_path, index=False)
    print(f"Archivo limpio guardado: {output_path}")

# Directorios de entrada y salida
input_dir = './Dataset_500_completo_v4'  # Cambia esto a la ruta de tus archivos completos
output_dir = './datos_limpios'  # Carpeta para los archivos limpios

# Crear directorio de salida si no existe
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Obtener todos los archivos completos
archivos = [f for f in os.listdir(input_dir) if f.endswith('_completo.csv')]
tickers = [f.split('_')[0] for f in archivos]

# Procesar cada ticker
for ticker in tickers:
    try:
        limpiar_datos(ticker, input_dir, output_dir)
    except Exception as e:
        print(f"Error procesando {ticker}: {str(e)}")

print("Proceso completado. Archivos limpios guardados en:", output_dir)

Archivo limpio guardado: ./datos_limpios/NWSA_completo_arreglado.csv
Archivo limpio guardado: ./datos_limpios/CF_completo_arreglado.csv
Archivo limpio guardado: ./datos_limpios/TSCO_completo_arreglado.csv
Archivo limpio guardado: ./datos_limpios/AJG_completo_arreglado.csv
Archivo limpio guardado: ./datos_limpios/GEN_completo_arreglado.csv
Archivo limpio guardado: ./datos_limpios/LOW_completo_arreglado.csv
Archivo limpio guardado: ./datos_limpios/HIG_completo_arreglado.csv
Archivo limpio guardado: ./datos_limpios/LYV_completo_arreglado.csv
Archivo limpio guardado: ./datos_limpios/XOM_completo_arreglado.csv
Archivo limpio guardado: ./datos_limpios/TEL_completo_arreglado.csv
Archivo limpio guardado: ./datos_limpios/PODD_completo_arreglado.csv
Archivo limpio guardado: ./datos_limpios/XYL_completo_arreglado.csv
Archivo limpio guardado: ./datos_limpios/EMR_completo_arreglado.csv
Archivo limpio guardado: ./datos_limpios/SBUX_completo_arreglado.csv
Archivo limpio guardado: ./datos_limpios/PEG_

In [8]:
import os
import pandas as pd

# --- Configuración ---
directorio = "./datos_limpios"  # Directorio con los archivos originales
archivo_referencia = "BX_completo_arreglado.csv"  # Archivo con las 111 columnas
directorio_salida = "./datos_normalizados"  # Directorio de salida

# Lista fija de columnas comunes
columnas_comunes = [
    "Date", "Total Liabilities Net Minority Interest", "Net Income From Continuing And Discontinued Operation",
    "Net Income Continuous Operations", "Share Issued", "Changes In Cash", "Total Debt", "Operating Revenue",
    "Net Income Common Stockholders", "Open", "Total Capitalization", "High", "Beginning Cash Position",
    "sectorKey", "Ordinary Shares Number", "Normalized Income", "Stockholders Equity", "Total Revenue",
    "Net Income Including Noncontrolling Interests", "Common Stock", "Low", "Net Income", "Diluted EPS",
    "Net Income From Continuing Operations", "Net Income From Continuing Operation Net Minority Interest",
    "Tangible Book Value", "Tax Effect Of Unusual Items", "Common Stock Equity", "Volume", "Pretax Income",
    "Cash And Cash Equivalents", "Basic Average Shares", "Long Term Debt And Capital Lease Obligation",
    "Investing Cash Flow", "End Cash Position", "Operating Cash Flow", "Close", "Total Assets", "Free Cash Flow",
    "Net Tangible Assets", "Total Equity Gross Minority Interest", "Basic EPS", "Capital Stock",
    "Diluted NI Availto Com Stockholders", "Financing Cash Flow", "Tax Rate For Calcs", "Diluted Average Shares",
    "Invested Capital"
]

# --- Crear directorio de salida si no existe ---
os.makedirs(directorio_salida, exist_ok=True)


# --- Función para normalizar los archivos ---
def normalizar_dataset(directorio, columnas_comunes, directorio_salida):
    archivos = [f for f in os.listdir(directorio) if f.endswith(".csv")]
    print(f"📂 Normalizando {len(archivos)} archivos...")

    for archivo in archivos:
        ruta_entrada = os.path.join(directorio, archivo)
        ruta_salida = os.path.join(directorio_salida, archivo)

        try:
            df = pd.read_csv(ruta_entrada)

            # Asegurar todas las columnas comunes
            for col in columnas_comunes:
                if col not in df.columns:
                    df[col] = 0  # Si falta, asigna 0

            # Mantener solo las columnas comunes y en el mismo orden
            df = df[columnas_comunes]

            df.to_csv(ruta_salida, index=False)
            print(f"✅ {archivo} normalizado y guardado")

        except Exception as e:
            print(f"⚠️ Error procesando {archivo}: {e}")


# --- Función para verificar consistencia ---
def verificar_consistencia(directorio):
    archivos = [f for f in os.listdir(directorio) if f.endswith(".csv")]
    columnas_base = None

    for archivo in archivos:
        df_temp = pd.read_csv(os.path.join(directorio, archivo), nrows=0)

        if columnas_base is None:
            columnas_base = set(df_temp.columns)
            print(f"🧩 Columnas base ({len(columnas_base)}): {sorted(columnas_base)}")
        else:
            columnas_actual = set(df_temp.columns)
            if columnas_base != columnas_actual:
                print(f"⚠️ {archivo} tiene columnas diferentes")
                print(f"Faltantes: {columnas_base - columnas_actual}")
                print(f"Adicionales: {columnas_actual - columnas_base}")

    print("✅ Verificación de consistencia completada")


# --- Ejecutar ---
normalizar_dataset(directorio, columnas_comunes, directorio_salida)
verificar_consistencia(directorio_salida)


📂 Normalizando 502 archivos...
✅ GOOGL_completo_arreglado.csv normalizado y guardado
✅ SRE_completo_arreglado.csv normalizado y guardado
✅ STT_completo_arreglado.csv normalizado y guardado
✅ PGR_completo_arreglado.csv normalizado y guardado
✅ LEN_completo_arreglado.csv normalizado y guardado
✅ TDY_completo_arreglado.csv normalizado y guardado
✅ VRSN_completo_arreglado.csv normalizado y guardado
✅ IVZ_completo_arreglado.csv normalizado y guardado
✅ MTCH_completo_arreglado.csv normalizado y guardado
✅ SYK_completo_arreglado.csv normalizado y guardado
✅ SW_completo_arreglado.csv normalizado y guardado
✅ NXPI_completo_arreglado.csv normalizado y guardado
✅ RTX_completo_arreglado.csv normalizado y guardado
✅ EXC_completo_arreglado.csv normalizado y guardado
✅ CARR_completo_arreglado.csv normalizado y guardado
✅ ADBE_completo_arreglado.csv normalizado y guardado
✅ CCL_completo_arreglado.csv normalizado y guardado
✅ LYV_completo_arreglado.csv normalizado y guardado
✅ IT_completo_arreglado.csv

In [9]:
# Verificar que todos los archivos tengan las mismas columnas
def verificar_consistencia(directorio):
    """
    Verifica que todos los archivos en el directorio tengan las mismas columnas.
    """
    archivos = [f for f in os.listdir(directorio) if f.endswith('.csv')]
    columnas_base = None
    
    for archivo in archivos:
        df_temp = pd.read_csv(os.path.join(directorio, archivo), nrows=0)
        
        if columnas_base is None:
            columnas_base = set(df_temp.columns)
            print(f"Columnas base ({len(columnas_base)}): {sorted(columnas_base)}")
        else:
            columnas_actual = set(df_temp.columns)
            if columnas_base != columnas_actual:
                print(f"Advertencia: {archivo} tiene columnas diferentes")
                print(f"Faltantes: {columnas_base - columnas_actual}")
                print(f"Adicionales: {columnas_actual - columnas_base}")
    
    print("Verificación de consistencia completada")

# Verificar consistencia
verificar_consistencia(directorio_salida)

Columnas base (48): ['Basic Average Shares', 'Basic EPS', 'Beginning Cash Position', 'Capital Stock', 'Cash And Cash Equivalents', 'Changes In Cash', 'Close', 'Common Stock', 'Common Stock Equity', 'Date', 'Diluted Average Shares', 'Diluted EPS', 'Diluted NI Availto Com Stockholders', 'End Cash Position', 'Financing Cash Flow', 'Free Cash Flow', 'High', 'Invested Capital', 'Investing Cash Flow', 'Long Term Debt And Capital Lease Obligation', 'Low', 'Net Income', 'Net Income Common Stockholders', 'Net Income Continuous Operations', 'Net Income From Continuing And Discontinued Operation', 'Net Income From Continuing Operation Net Minority Interest', 'Net Income From Continuing Operations', 'Net Income Including Noncontrolling Interests', 'Net Tangible Assets', 'Normalized Income', 'Open', 'Operating Cash Flow', 'Operating Revenue', 'Ordinary Shares Number', 'Pretax Income', 'Share Issued', 'Stockholders Equity', 'Tangible Book Value', 'Tax Effect Of Unusual Items', 'Tax Rate For Calcs', 

In [10]:
 !pip install TA-Lib

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [11]:
import pandas as pd
import glob

def obtener_sectores_unicos(path_pattern="data/*.csv", col_sector="sectorKey"):
    """
    Lee múltiples CSV y devuelve una lista de todos los sectores únicos.
    
    Args:
        path_pattern (str): patrón de búsqueda para archivos CSV (ej: 'data/*.csv')
        col_sector (str): nombre de la columna que contiene el sector
    
    Returns:
        list: lista de sectores únicos encontrados en todos los archivos
    """
    sectores = set()
    
    # Buscar todos los CSV en la ruta indicada
    for file in glob.glob(path_pattern):
        df = pd.read_csv(file)
        if col_sector in df.columns:
            sectores.update(df[col_sector].dropna().unique())
    
    return sorted(list(sectores))

# Ejemplo de uso
sectores_totales = obtener_sectores_unicos("./datos_normalizados/*.csv", col_sector="sectorKey")
print("Sectores encontrados:", sectores_totales)

Sectores encontrados: ['basic-materials', 'communication-services', 'consumer-cyclical', 'consumer-defensive', 'energy', 'financial-services', 'healthcare', 'industrials', 'real-estate', 'technology', 'utilities']


In [12]:
len(sectores_totales)

11

In [13]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
import joblib
import talib as ta


def calculate_technical_indicators(df):
    """Calcula 48 indicadores técnicos y agrega datos financieros"""
    # Precios
    open_price = df['Open'].values
    high = df['High'].values
    low = df['Low'].values
    close = df['Close'].values
    volume = df['Volume'].values
    
    features = {}
    
    # ===== INDICADORES TÉCNICOS (48 features) =====
    
    # Medias móviles (8 features)
    features['SMA_5'] = ta.SMA(close, timeperiod=5)
    features['SMA_10'] = ta.SMA(close, timeperiod=10)
    features['SMA_20'] = ta.SMA(close, timeperiod=20)
    features['SMA_50'] = ta.SMA(close, timeperiod=50)
    features['EMA_5'] = ta.EMA(close, timeperiod=5)
    features['EMA_10'] = ta.EMA(close, timeperiod=10)
    features['EMA_20'] = ta.EMA(close, timeperiod=20)
    features['EMA_50'] = ta.EMA(close, timeperiod=50)
    
    # Osciladores (12 features)
    features['RSI'] = ta.RSI(close)
    features['STOCH_K'], features['STOCH_D'] = ta.STOCH(high, low, close)
    features['MACD'], features['MACD_Signal'], features['MACD_Hist'] = ta.MACD(close)
    features['ADX'] = ta.ADX(high, low, close)
    features['CCI'] = ta.CCI(high, low, close)
    features['WILLR'] = ta.WILLR(high, low, close)
    features['ULTOSC'] = ta.ULTOSC(high, low, close)
    features['ROC'] = ta.ROC(close)
    
    # Volatilidad (6 features)
    features['ATR'] = ta.ATR(high, low, close)
    features['NATR'] = ta.NATR(high, low, close)
    features['TRANGE'] = ta.TRANGE(high, low, close)
    features['BB_Upper'], features['BB_Middle'], features['BB_Lower'] = ta.BBANDS(close)
    
    # Volumen (5 features)
    # features['OBV'] = ta.OBV(close, volume)
    # features['AD'] = ta.AD(high, low, close, volume)
    # features['ADOSC'] = ta.ADOSC(high, low, close, volume)
    # features['MFI'] = ta.MFI(high, low, close, volume)
    
    # Momentum (8 features)
    features['MOM'] = ta.MOM(close)
    features['PPO'] = ta.PPO(close)
    features['APO'] = ta.APO(close)
    features['AROON_Up'], features['AROON_Down'] = ta.AROON(high, low)
    features['AROONOSC'] = ta.AROONOSC(high, low)
    features['BOP'] = ta.BOP(open_price, high, low, close)
    
    # Precios relativos (9 features)
    features['Price_Change'] = close / np.roll(close, 1) - 1
    features['High_Low_Ratio'] = high / low
    features['Close_Open_Ratio'] = close / open_price
    # features['Volume_Change'] = volume / np.roll(volume, 1) - 1
    

    # ===== RATIOS FINANCIEROS DERIVADOS =====
    with np.errstate(divide='ignore', invalid='ignore'):
        features['ROE'] = df['Net Income'] / df['Total Equity Gross Minority Interest']
        features['ROA'] = df['Net Income'] / df['Total Assets']
        features['Debt_to_Equity'] = df['Total Debt'] / df['Total Equity Gross Minority Interest']
        features['Profit_Margin'] = df['Net Income'] / df['Total Revenue']
        features['Operating_Margin'] = df['Operating Revenue'] / df['Total Revenue']
        features['Asset_Turnover'] = df['Total Revenue'] / df['Total Assets']
        features['Financial_Leverage'] = df['Total Assets'] / df['Common Stock Equity']
        features['EPS_Calc'] = df['Net Income'] / df['Diluted Average Shares']



    # ===== DATOS FINANCIeros DEL CSV (46 features adicionales) Comparado con el CLOSE=====
    


    # Flujo de caja y liquidez
    features['Free_Cash_Flow'] = df['Free Cash Flow']/close
    features['Operating_Cash_Flow'] = df['Operating Cash Flow']/close
    features['Investing_Cash_Flow'] = df['Investing Cash Flow']/close
    features['Financing_Cash_Flow'] = df['Financing Cash Flow']/close
    features['Beginning_Cash_Position'] = df['Beginning Cash Position']/close
    features['End_Cash_Position'] = df['End Cash Position']/close
    features['Changes_In_Cash'] = df['Changes In Cash']/close
    features['Cash_And_Cash_Equivalents'] = df['Cash And Cash Equivalents']/close
    
    # Rentabilidad
    features['Net_Income'] = df['Net Income']/close
    features['Pretax_Income'] = df['Pretax Income']/close
    features['Total_Revenue'] = df['Total Revenue']/close
    features['Operating_Revenue'] = df['Operating Revenue']/close
    features['Normalized_Income'] = df['Normalized Income']/close
    features['Net_Income_Common_Stockholders'] = df['Net Income Common Stockholders']/close
    features['Net_Income_From_Continuing_Operations'] = df['Net Income From Continuing Operations']/close
    features['Net_Income_From_Continuing_And_Discontinued_Operation'] = df['Net Income From Continuing And Discontinued Operation']/close
    features['Net_Income_Including_Noncontrolling_Interests'] = df['Net Income Including Noncontrolling Interests']/close
    features['Net_Income_Continuous_Operations'] = df['Net Income Continuous Operations']/close
    features['Diluted_NI_Availto_Com_Stockholders'] = df['Diluted NI Availto Com Stockholders']/close
    features['Net_Income_From_Continuing_Operation_Net_Minority_Interest'] = df['Net Income From Continuing Operation Net Minority Interest']/close
    
    # Por acción
    features['Basic_EPS'] = df['Basic EPS']/close
    features['Diluted_EPS'] = df['Diluted EPS']/close
    
    # Acciones
    features['Diluted_Average_Shares'] = df['Diluted Average Shares']/close
    features['Basic_Average_Shares'] = df['Basic Average Shares']/close
    features['Share_Issued'] = df['Share Issued']/close
    features['Ordinary_Shares_Number'] = df['Ordinary Shares Number']/close
    features['Capital_Stock'] = df['Capital Stock']/close
    features['Common_Stock'] = df['Common Stock']/close
    
    # Balance general
    features['Total_Assets'] = df['Total Assets']/close
    features['Total_Liabilities_Net_Minority_Interest'] = df['Total Liabilities Net Minority Interest']/close
    features['Total_Equity_Gross_Minority_Interest'] = df['Total Equity Gross Minority Interest']/close
    features['Common_Stock_Equity'] = df['Common Stock Equity']/close
    features['Stockholders_Equity'] = df['Stockholders Equity']/close
    features['Total_Capitalization'] = df['Total Capitalization']/close
    features['Invested_Capital'] = df['Invested Capital']/close
    features['Net_Tangible_Assets'] = df['Net Tangible Assets']/close
    features['Tangible_Book_Value'] = df['Tangible Book Value']/close
    
    # Deuda
    features['Total_Debt'] = df['Total Debt']/close
    features['Long_Term_Debt_And_Capital_Lease_Obligation'] = df['Long Term Debt And Capital Lease Obligation']/close
    
    # Ratios financieros
    features['Tax_Rate_For_Calcs'] = df['Tax Rate For Calcs']/close
    features['Tax_Effect_Of_Unusual_Items'] = df['Tax Effect Of Unusual Items']/close
    features['Close'] = df['Close']
    features['Date'] = df['Date']
    
    # Sector (convertir a numérico si es necesario)
    # if 'sectorKey' in df.columns:
    #     # One-hot encoding para sector
    #     sector_dummies = pd.get_dummies(df['sectorKey'], prefix='Sector')
    #     for col in sector_dummies.columns:
    #         features[col] = sector_dummies[col]
    if 'sectorKey' in df.columns:
        # Generar one-hot encoding SOLO de los sectores presentes en este df
        sector_dummies = pd.get_dummies(df['sectorKey'], prefix='Sector')

        # Crear todas las columnas esperadas en base a sectores_totales
        expected_cols = [f"Sector_{s}" for s in sectores_totales]

        # Reindexar para incluir todos los sectores, rellenando con 0 donde falten
        sector_dummies = sector_dummies.reindex(columns=expected_cols, fill_value=0)

        # Convertir booleanos a enteros (0 o 1)
        sector_dummies = sector_dummies.astype(int)

        # Agregar cada columna dummy al diccionario de features
        for col in sector_dummies.columns:
            features[col] = sector_dummies[col]  
    
    # Convertir a DataFrame
    features_df = pd.DataFrame(features, index=df.index)
    
    # Eliminar columnas que puedan tener todos valores NaN
    features_df = features_df.dropna(axis=1, how='all')
    
    # Llenar valores NaN restantes con 0 o método apropiado
    features_df = features_df.fillna(method='ffill').fillna(method='bfill').fillna(0)
    
    return features_df


In [14]:
def create_target_variable(df, horizon):
    """Crea la variable objetivo: % cambio en los próximos 3 meses (63 días trading)"""
    # Asegurarse de que los datos están ordenados por fecha
    df = df.sort_index()
    
    # Calcular precio futuro (3 meses después)
    future_prices = df['Close'].shift(-horizon)
    current_prices = df['Close']
    
    # Calcular porcentaje de cambio
    target = ((future_prices - current_prices) / current_prices) * 100
    
    # Crear DataFrame con target
    target_df = pd.DataFrame({
        'Target_3M_Return': target,
        'Target_Direction': np.where(target > 0, 1, 0)  # 1: subida, 0: bajada
    }, index=df.index)
    df_with_target = df.join(target_df, how='left')
    
    # Eliminar valores donde no hay futuro disponible
    df_with_target = df_with_target.dropna()
    
    return df_with_target

In [15]:
def data_augmentation_dataset(directorio, directorio_salida=None):
    """
    Normaliza todos los archivos CSV en el directorio manteniendo solo las columnas comunes.
    """
    if directorio_salida is None:
        directorio_salida = directorio
    
    # Crear directorio de salida si no existe
    if not os.path.exists(directorio_salida):
        os.makedirs(directorio_salida)
    
    # Listar todos los archivos CSV en el directorio
    archivos = [f for f in os.listdir(directorio) if f.endswith('.csv')]
    # archivos = ["A_completo_arreglado.csv"]
    
    for archivo in archivos:
        try:
            # Leer el archivo
            df = pd.read_csv(os.path.join(directorio, archivo))
            df_2 = calculate_technical_indicators(df=df)
            df_3 = create_target_variable(df=df_2, horizon= 60)

            # Guardar el archivo normalizado
            ruta_salida = os.path.join(directorio_salida, archivo)
            df_3.to_csv(ruta_salida, index=False)
            
        except Exception as e:
            print(f"Error procesando {archivo}: {str(e)}")
            continue


In [16]:
import os
import shutil

# Configuración
directorio = "./datos_normalizados"  # Cambia esto por tu directorio
directorio_salida = "./data_final_v3"  # Directorio para archivos normalizados

# Encontrar columnas comunes
data_augmentation_dataset(directorio, directorio_salida)

/var/folders/gr/k4cy24rj4118z6hjbdmg2hdw0000gn/T/ipykernel_22800/809798560.py:173: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  features_df = features_df.fillna(method='ffill').fillna(method='bfill').fillna(0)
/var/folders/gr/k4cy24rj4118z6hjbdmg2hdw0000gn/T/ipykernel_22800/809798560.py:173: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  features_df = features_df.fillna(method='ffill').fillna(method='bfill').fillna(0)
/var/folders/gr/k4cy24rj4118z6hjbdmg2hdw0000gn/T/ipykernel_22800/809798560.py:173: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  features_df = features_df.fillna(method='ffill').fillna(method='bfill').fillna(0)
/var/folders/gr/k4cy24rj4118z6hjbdmg2hdw0000gn/T/ipykernel_22800/809798560.py:173: FutureWa

In [17]:
def create_target_variable_v2(df, horizon):
    """Crea la variable objetivo: % cambio en los próximos 3 meses (63 días trading)"""
    # Asegurarse de que los datos están ordenados por fecha
    df = df.sort_index()
    
    # Calcular precio futuro (3 meses después)
    future_prices = df['Close'].shift(-horizon)
    current_prices = df['Close']
    
    # Calcular porcentaje de cambio
    target = ((future_prices - current_prices) / current_prices) * 100
    
    # Crear DataFrame con target
    target_df = pd.DataFrame({
        'Target_3M_Return': target,
        'Target_Direction': np.where(target > 0, 1, 0)  # 1: subida, 0: bajada
    }, index=df.index)
    df_with_target = df.join(target_df, how='left')
    
    # Eliminar valores donde no hay futuro disponible
    # df_with_target = df_with_target.dropna()
    
    return df_with_target

In [18]:
def data_augmentation_dataset_with_unknow_values(directorio, directorio_salida=None):
    """
    Normaliza todos los archivos CSV en el directorio manteniendo solo las columnas comunes.
    """
    if directorio_salida is None:
        directorio_salida = directorio
    
    # Crear directorio de salida si no existe
    if not os.path.exists(directorio_salida):
        os.makedirs(directorio_salida)
    
    # Listar todos los archivos CSV en el directorio
    archivos = [f for f in os.listdir(directorio) if f.endswith('.csv')]
    # archivos = ["A_completo_arreglado.csv"]
    
    for archivo in archivos:
        try:
            # Leer el archivo
            df = pd.read_csv(os.path.join(directorio, archivo))
            df_2 = calculate_technical_indicators(df=df)
            df_3 = create_target_variable_v2(df=df_2, horizon= 60)

            # Guardar el archivo normalizado
            ruta_salida = os.path.join(directorio_salida, archivo)
            df_3.to_csv(ruta_salida, index=False)
            
        except Exception as e:
            print(f"Error procesando {archivo}: {str(e)}")
            continue

In [19]:
import os
import shutil
import pandas as pd

# Configuración
directorio = "./datos_normalizados"  # Cambia esto por tu directorio
directorio_salida = "./data_final_unknow_values_v3"  # Directorio para archivos normalizados

# Encontrar columnas comunes
data_augmentation_dataset_with_unknow_values(directorio, directorio_salida)

/var/folders/gr/k4cy24rj4118z6hjbdmg2hdw0000gn/T/ipykernel_22800/809798560.py:173: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  features_df = features_df.fillna(method='ffill').fillna(method='bfill').fillna(0)
/var/folders/gr/k4cy24rj4118z6hjbdmg2hdw0000gn/T/ipykernel_22800/809798560.py:173: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  features_df = features_df.fillna(method='ffill').fillna(method='bfill').fillna(0)
/var/folders/gr/k4cy24rj4118z6hjbdmg2hdw0000gn/T/ipykernel_22800/809798560.py:173: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  features_df = features_df.fillna(method='ffill').fillna(method='bfill').fillna(0)
/var/folders/gr/k4cy24rj4118z6hjbdmg2hdw0000gn/T/ipykernel_22800/809798560.py:173: FutureWa